In [4]:
# definicion de palabras clave
palabras_clave = ['Creatin', 'creatin']


# Instalacion
!pip install tika
import pandas as pd
import numpy as np

# Correccion
from tika import parser
raw = parser.from_file('Mayorista.pdf')
df = (raw['content'])
df = df.replace('\n',',')
df = df.replace(',,',',')
df = df.replace(',,,,,,,,,,,,,,,,,,,,','')

# Reemplazo de las comas en los precios (para luego dividir los productos por coma)
for i,v in enumerate(df):
  if i < len(df)-3:
    if v == ',':
      if df[i+2] == '0':
        df = df[:i] + '.' + df[i + 1:]

      try:
        valor1 = int(df[i+1])
        if type(valor1) == int:
          df = df[:i] + '.' + df[i + 1:]
      except:
        continue

      try:
        valor2 = int(df[i+2])
        if type(valor2) == int:
          df = df[:i] + '.' + df[i + 1:]
      except:
          continue


# Split
df = df.split(',')

# lista de precios.
total_precios = []
for i1,v1 in enumerate(df):
  for i2,v2 in enumerate(v1):
    if v2 == '$':
      total_precios.append(int(str(round(int(df[i1][i2+2:])))))

# Agregar ganancia
df1 = df.copy()
for i1,v1 in enumerate(df1):
  for i2,v2 in enumerate(v1):
    if v2 == '$':
      size = len(df1[i1])
      to_delete = len(df1[i1][i2+2:])
      mod_string = df1[i1][:size - to_delete]
      old_price = (round(int(df1[i1][i2+2:])))
      multiplicador = 0
      porcentaje_ganancia = 0

      # Ganancia segun decil
      ganancias = [0.50, 0.53, 0.56, 0.59, 0.62, 0.65, 0.69, 0.72, 0.75, 0.8]
      percentiles_precios = np.percentile(total_precios, np.arange(0, 100, 10)).tolist()
      porcentaje_ganancia = 0
      for i in range(len(percentiles_precios)):
        try:
          if old_price in range(int(percentiles_precios[i]), int(percentiles_precios[i+1])):
            porcentaje_ganancia = ganancias[i]
            break
        except:
          if old_price >= percentiles_precios[-1]:
            porcentaje_ganancia = ganancias[-1]
            break
          else:
            print(f'Precio {old_price} no se ubica en percentiles')
            break

      mod_string += str(round(old_price / porcentaje_ganancia))
      df1[i1] = mod_string
df = df1.copy()

# Quitar filas irrelevantes
for i,v in enumerate(df):
  if v.startswith('Fecha') or v.startswith('Página') or v.startswith('MARCA') or v.startswith('Lista'):
     df[i] = ''
  for palabra in palabras_clave:
    if palabra in v:
      valor = v
      break
    else:
      valor = ''
  df[i] = valor

# Eliminado de espacios vacios
while '' in df:
  df.remove('')

# Insertar marca
df1 = df.copy()
corrector = 0
for i,v in enumerate(df):

  # Obtener marca
  for i2,v2 in enumerate(v):
    try:
      int(v2)
      coordenada = i2
      break
    except:
      continue
  marca = v[0:coordenada-1]
  
  # Comparacion con la marca anterior
  try:
    if marca == df[i-1][0:len(marca)]:
      continue
    else:
      if i > 0:
        # Inserto marca
        df1.insert(i+corrector, '\n')
        df1.insert(i+1+corrector, ' '*10 + marca)
        corrector = corrector + 2
      else:
        df1.insert(i+corrector, ' '*10 + marca)
        corrector = corrector + 1
        continue
        
  except:
    continue

df = df1.copy() 

# Encabezado piola
df.insert(0, 'ROYAL DISTRIBUIDORA \n \n  LISTA MINORISTA DE CREATINAS \n \n \n')

# Formato prolijo
n = '\n'.join(df)

# Guardado
text_file = open("Lista Minorista.txt", "w")
text_file.write(n)
text_file.close()
from google.colab import files
files.download('Lista Minorista.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>